In [4]:
#################################################################################################
# Utilizing Prof. Corada's code (to get to the inputs for Locality Sensitive Hashing functions)
#################################################################################################

#################################################################################################
# Processing the article DB and creating shingles
#################################################################################################

import string
from binascii import crc32

def _read_data(filepath):
    with open(filepath, 'r') as f:
        result = []
        for line in f:
            first_space = line.find(' ')
            article_id = line[:first_space]
            text = line[(first_space+1):]
            item = (article_id, text)
            result.append(item)
    return result

def _process_articles(articles):
    _punct_table = str.maketrans(dict.fromkeys(string.punctuation))
    
    def _process_one(x):
        docid, text = x
        return (docid, text.strip() \
                        .translate(_punct_table) \
                        .lower() \
                        .replace(' ', ''))
    
    return [_process_one(x) for x in articles]                   

def _shingle_document(text, k):
    shingles = set()
    n = len(text)
    for i in range(0, n-k):
        shingle = text[i:(i+k)]
        hashed_shingle = crc32(shingle.encode('utf-8')) & 0xffffffff
        shingles.add(hashed_shingle)
    return shingles

class ArticleDB:
    def __init__(self, filename):
        self._filename = filename
        self._articles = _read_data(self._filename)
        self._processed_articles = _process_articles(self._articles)
        self._docids = [docid for docid, _ in self._processed_articles]
        
    def shingle_data(self, k):
        def _shingle_one(x):
            docid, text = x
            sharded_doc = _shingle_document(text, k)
            return (docid, sharded_doc)
        
        return [_shingle_one(x) for x in self._processed_articles]

#################################################################################################
# Computing Jaccard Similarity
#################################################################################################

def _jaccard_similarity(s1, s2):
    union_size = len(s1.union(s2))
    intersection_size = len(s1.intersection(s2))
    return 1. * intersection_size / union_size

class Jaccard:
    def __init__(self):
        self._jaccard_dict = None
        
    def compute_similarity(self, shingled_data, docids=None):
        if docids is not None:
            shingled_data = [x for x in shingled_data if x[0] in docids]
        
        ndocs = len(shingled_data)
        self._jaccard_dict = dict()
        
        for i in range(ndocs-1):
            for j in range(i+1, ndocs):
                (doci, si) = shingled_data[i]
                (docj, sj) = shingled_data[j]
                
                key = tuple(sorted((doci, docj)))
                js = _jaccard_similarity(si, sj)
                self._jaccard_dict[key] = js
        
    def get_similarity(self, doci, docj):
        key = tuple(sorted((doci, docj)))
        return self._jaccard_dict[key]
    
#################################################################################################
# Part 1 - Main code, calling the functions; Running the experiment which yields k=10
#################################################################################################

import pandas as pd
import random
import numpy as np
from plagiarism_lib.ArticleDB import ArticleDB
from plagiarism_lib.jaccard import Jaccard

def _setup_fp_dataset(artdb, truth_file):
    with open(truth_file, 'r') as f:
        truth_pairs = [tuple(sorted(line.strip().split()))
                       for line in f]
        
        true_articles = set()
        for s1, s2 in truth_pairs:
            true_articles.add(s1)
            true_articles.add(s2)
        
        alldocs = set(artdb._docids)
        false_candidates = alldocs.difference(true_articles)
        false_partners = random.sample(false_candidates, len(true_articles))
        false_pairs = list(zip(true_articles, false_partners))
        
        alldocs = list(true_articles) + false_partners
        return truth_pairs, false_pairs, alldocs

def _get_kstats(truth_pairs, false_pairs, jacc):
    true_sims = [jacc.get_similarity(x,y) for x,y in truth_pairs]
    false_sims = [jacc.get_similarity(x,y) for x,y in false_pairs]
    
    tp = np.mean(true_sims)
    fp = np.mean(false_sims)
    
    return tp, fp
       
def run_experiment(train_file, truth_file,
                   kvals = [2,5,10,20,40,60,120]):
    artdb = ArticleDB(train_file)
    truth_pairs, false_pairs, alldocs = _setup_fp_dataset(artdb, truth_file)
        
    tp = []
    fp = []
    
    for k in kvals:
        print("Processing data for k=", k)
        shingled_data = artdb.shingle_data(k)
        
        
        jacc = Jaccard()
        jacc.compute_similarity(shingled_data, docids=alldocs)
        
        ktp, kfp = _get_kstats(truth_pairs, false_pairs, jacc)
        tp.append(ktp)
        fp.append(kfp)
     
    df = pd.DataFrame({'k': kvals, 'sim_true': tp, 'sim_false': fp})
    return df

#################################################################################################
# Creating Hash functions for Min Hash and creating the vector of hashes for LSH
#################################################################################################

import random

DEFAULT_P = 2**33-355
DEFAULT_M = 4294967295

def _make_hash(p=DEFAULT_P, m=DEFAULT_M):
    a = random.randint(1, p-1)
    b = random.randint(0, p-1)
    return lambda x: ((a * x + b) % p) % m

def _make_hashes(n):
    return [_make_hash() for _ in range(n)]


def _make_vector_hash(n, m=DEFAULT_M):
    hfuncs = _make_hashes(n)
    def _f(x):
        acc = 0
        for i in range(len(x)):
            h = hfuncs[i]
            acc += h(x[i])
        return acc % m
    return _f

#################################################################################################
# Part 2 - Inverting Shingles and Code to create MinHash Signature Matrix
#################################################################################################

import numpy as np
import _make_hashes

# Invert shingled article dataset so we can iterate over shingles
#
# shingled_data: list of (docid, shingles) tuples
#    docid: document id 
#    shingles: set of 32-bit integers encoding document shingles of corresponding document
#
# returns: sorted list of tuples (s, docid):
#    s: 32-bit integer encoding document shingle
#    docid: id of document where s occurs
def invert_shingles(shingled_data):
    inv_index = []
    docids = []
    
    for (docid, shingles) in shingled_data:
        docids.append(docid)
        for s in shingles:
            item = (s, docid)
            inv_index.append(item)
    return sorted(inv_index), docids


# Create a minhash signature matrix
#
# shingled_data: a shingled article dataset, format depends on argument 'inverted' (see below)
# num_hashes: number of hash functions to use in the minhash summary
# inverted: boolean, is the shingled data already inverted (see below)
#
# returns: tuple (mh, docids) 
#   mh: a numpy matrix (num_hashes x num_docs) of minhash signature
#   docids: a list of document ids
# 
# note: if argument 'inverted' is True, then shingled data is already sorted by shingle, so
# the minhash algorithm can use directly. In this case, 'shingled_data' is assumed to be 
# a sorted list of (s, docid) tuples, with s a shingle, and docid a document id/
#
# If argument 'inverted' is False, then shingled data is organized by document and we need
# to create an inverted index so we can iterate by shingle. In this case 'shingled_data' is
# assumed to be a list of (docid, shingles) tuples with docid a document id and
# shingles a 'set' of shingles
def _make_minhash_sigmatrix(shingled_data, num_hashes, inverted=False):
    
    # invert the shingled data if necessary
    if inverted:
        inv_index, docids = shingled_data
    else:
        inv_index, docids = invert_shingles(shingled_data)
        
    # initialize the signature matrix with infinity in every entry
    num_docs = len(docids)
    sigmat = np.full([num_hashes, num_docs], np.inf)       # create a matrix = num_hashes x num_docs
    
    # create num_hashes random hash functions
    hash_funcs = _make_hashes(num_hashes)                   # yields a list of 'num_hashes' number of hash functions
    
    # iterate over shingles 
    for s, docid in inv_index:
        # evaluate hash functions on shingle
        hashvals = [h(s) for h in hash_funcs]  # yields a list of hashed values of shingle s, passed to each of hash functions 
        
        # find the appropriate column in signature matrix for document id
        j = docids.index(docid)
        
        # update rows of appropriate column as needed
        for i in range(num_hashes):
            if hashvals[i] < sigmat[i,j]:      # logic yields the minimum value of shingle s evaluated from all hash functions
                # saw a smaller hash value, update the signature matrix
                sigmat[i,j] = hashvals[i]
    return sigmat, docids


# Objects used to create and query minhash summaries
# Example using 10 hash functions:
#   mh = MinHash(10) 
#   mh.make_matrix(article_db)
#   mh.get_similarity(doci, docj)
class MinHash:
    # Construct an empty object that will use 'num_hashes' in the summary
    #
    # num_hashes: number of hashes to use in the 
    def __init__(self, num_hashes):
        self._num_hashes = num_hashes
        self._docids = None
        self._mat = None
        
    # Create the signature matrix
    # 
    # shingled_data: a shingled document dataset (see _make_minhash_sigmatrix)
    # inverted: is the dataset already inverted (see _make_minhash_sigmatrix)
    #
    # side effect: updates self._docids and self._mat atttributes of object
    def make_matrix(self, shingled_data, inverted=False):
        self._mat, self._docids = _make_minhash_sigmatrix(shingled_data,
                                                          self._num_hashes,
                                                          inverted=inverted)
        
    # compute minhash JS estimate for documents di and dj
    #
    # di: id of first document
    # dj: id of second document
    #
    # returns: minhash JS estimate (double)
    def get_similarity(self, di, dj):
        i = self._docids.index(di)
        j = self._docids.index(dj)
        return np.mean(self._mat[:,i] == self._mat[:,j])
    

#################################################################################################
# Main Code for Part 2 - Calling Functions
#################################################################################################

import pandas as pd
import numpy as np

from plagiarism_lib.jaccard import Jaccard
from plagiarism_lib.minhash import MinHash, invert_shingles

## 
# Create a pandas DataFrame containing exact Jaccard similarities
# between pairs of documents
#
# exp_data: a shingled article dataset
# docids: ids for documents in 'exp_data'
#
# returns: a pandas DataFrame with columns:
#   doci: id of first document
#   docj: id of second document
#   js: Jaccard similarity between pair of documents
def make_js_df(exp_data, docids):
    # create a Jaccard object and compute pairwise similarities
    jacc = Jaccard()
    jacc.compute_similarity(exp_data)

    # create the data frame with all pairwise Jaccard similarities
    ndocs = len(docids)
    doci_series = []
    docj_series = []
    js_series = []
    
    for i in range(ndocs-1):
        for j in range(i+1, ndocs):
            di = docids[i]
            dj = docids[j]
            s = jacc.get_similarity(di, dj)
            doci_series.append(di)
            docj_series.append(dj)
            js_series.append(s)
            
    return pd.DataFrame({'doci': doci_series, 
                         'docj': docj_series, 
                         'js': js_series})


##
# Run an experiment to measure error of MinHash Jaccard similarity estimates
# for various number of hash functions used in MinHash summaries
#
# exp_data: a shingled article dataset
# exp_df: a pandas DataFrame with exact JS computed between pairs of articles 
# hash_vals: number of hash functions to use in the various MinHash JS estimates
#
# side effect: adds columns to exp_df:
#   mh_<k>: Minhash estimate of JS for pair of documents using k hash functions
def run_mh_exp(exp_data, exp_df, hash_vals=[10,20,50,100,1000]):
    
    # invert the shingled articles so we can iterate over
    # shingles 
    inv_data = invert_shingles(exp_data)
    
    # for each of the number of hashes used in experiment
    for num_hash in hash_vals:
        print("Doing minhash for ", num_hash, " hashes")
        
        # compute minhahs signature matrix
        mh = MinHash(num_hash)
        mh.make_matrix(inv_data, inverted=True)
        
        # add the minhash JS estimates for given number of hash functions
        # to experiment dataset
        cur_series = []
        for _, row in exp_df.iterrows():
            s = mh.get_similarity(row['doci'], row['docj'])
            cur_series.append(s)
            
        series_name = 'mh_' + str(num_hash)
        exp_df[series_name] = cur_series
  
# Compute root mean squared error between Minhash estimates and exact JS
# 
# exp_df: pandas DataFrame with exact JS and minhash JS similarities (created by run_mh_exp)
# hash_vals: the hash values used in the various minhash estimates
#
# returns: a pandas DataFrame with columns
#    h: hash value used in minhash estimate
#    rmse: root mean squared error of minhash estimate
def post_process_df(exp_df, hash_vals):
    tmp_df = exp_df
    for num_hash in hash_vals:
        mh_series = 'mh_' + str(num_hash)
        series_name = 'diff_' + str(num_hash)
        tmp_df[series_name] = np.square(exp_df['js'] - exp_df[mh_series])
    
    cols = ['diff_' + str(num_hash) for num_hash in hash_vals]
    mns = tmp_df[cols].mean()
    rmse_df = pd.DataFrame({'h': hash_vals, 'rmse': np.sqrt(mns)})
    return rmse_df

#################################################################################################
# Start of Part 3 - Function to get the number of Bands in LSH algorithm (based on optimization of error function)
#################################################################################################

import scipy.optimize as opt
import math

# Choose number of bands for LSH based on threshold t and number of rows
# in minhash signature matrix n
#
# t: desired threshold for candidate detection (double in (0,1))
# n: number of rows in minhash signature matrix
#
# returns: tuple (b, final_t)
#    b: number of bands (integer such that b*r = n)
#    final_t: the final threshold t = (1/b)^(1/r) s.t. b*r=n
#
# Uses Nelder-Mead optimization method to find value b such that
# (1/b)^(b/n) is closest (in least squares) value possible to input threshold t
#
# Example:
#   b, final_t = _choose_nbands(.8, 100)
def _choose_nbands(t, n):
    def _error_fun(x):
        cur_t = (1/x[0])**(x[0]/n)
        return (t-cur_t)**2
    
    opt_res = opt.minimize(_error_fun, x0=(10), method='Nelder-Mead')
    b = int(math.ceil(opt_res['x'][0]))
    r = int(n / b)
    final_t = (1/b)**(1/r)
    return b, final_t

#################################################################################################
# Implementing LSH
#################################################################################################

#################################################################################################
# Part 1 - Do_LSH function
#################################################################################################

from collections import defaultdict

def do_lsh(minhash_sigmatrix, numhashes, docids, threshold):
  # choose the number of bands, and rows per band to use in LSH
  b, _ = _choose_nbands(threshold, numhashes)
  r = int(numhashes / b)          # Number of rows in each band

  narticles = len(docids)

  # generate a random hash function that takes vectors of length r as input
  hash_func = _make_vector_hash(r)

  # setup the list of hashtables, will be populated with one hashtable per band
  buckets = []

  # fill hash tables for each band
  for band in range(b):
    # figure out which rows of minhash signature matrix to hash for this band
    start_index = int(band * r)
    end_index = min(start_index + r, numhashes)

    # initialize hashtable for this band
    cur_buckets = defaultdict(list)
    
    for j in range(narticles):                          # to access each doc ID from sigmatrix
        m = []                                          # List to collect hash values for band 'b' for document 'j'
        for k in range(start_index, end_index+1):       # this loop will extract the rows from sigmatrix
            m.append(minhash_sigmatrix[k,j])            # Collecting the minhash values from sigmatrix
        hashval = hash_func(m)                          # Computing LSH hash value by passing the list of minhash values
        cur_buckets[hashval].append(docids[j])          # Capturing the document ids, and indexing it with the hashvalue
    
    # add this hashtable to the list of hashtables
    buckets.append(cur_buckets)

  return buckets

#################################################################################################
# Part 2 - Finding candidate similar article pairs
#################################################################################################

def find_candidate_pair(buckets):
    candidate_pair = []
    hashvalues = []
    docid = []
    for b in buckets:                          # for each bucket, we know similar docs will hash to the same bucket
        for hval,id in b.items():              # extracting hash values, list of document IDs
            hashvalues.append(hval)
            docid.append(id)                   # is a nested list of doc IDs
        docid = tuple(docid)
        for i in range(len(docid)):
            id1 = []
            id1.append(docid[i])
            for j in range(i+1,len(docid)):
                id2 = []
                id2.append(docid[j])
                tup = tuple(id1) + tuple(id2)
                candidate_pair.append(tup)     # creating the list of tuples with candidate pairs of docs
    return candidate_pair

#################################################################################################
# Experiment 2 - Computing Precision and Recall
#################################################################################################

import pandas as pd
def experiment2_LSH(candidate_pair, threshold):
    mainfile = r'C:\Vidit\PhD\Fall 2018\CMSC 643 - Hector\Project 1\TestFile10000.txt'  #Getting 10000 file
    with open(mainfile, 'r') as f:
        all_pairs = [tuple(sorted(line.strip().split()))
                       for line in f]
    total_articles = set()
    for s1, s2 in all_pairs:
        total_articles.add(s1)
        total_articles.add(s2)
    truth_file = r'C:\Vidit\PhD\Fall 2018\CMSC 643 - Hector\Project 1\TruthFile10000.txt'  #Getting 10000 truth file
    with open(truth_file, 'r') as f:
        truth_pairs = [tuple(sorted(line.strip().split()))
                       for line in f]
    true_articles = set()
    for s1, s2 in truth_pairs:
        true_articles.add(s1)
        true_articles.add(s2)
    threshold = input("Enter the threshold value for similarity:")
    numhashes = input("Enter the no of hash functions:")
    buckets = do_lsh(minhash_sigmatrix, numhashes, docids, threshold)
    candidate_pair = find_candidate_pair(buckets)
    result = []
    true_positive = false_positive = false_negative = true_negative = 0
    for id1,id2 in total_articles:                              # accessing pairs of doc ids in 10000 dataset
        for i,j in candidate_pair:                              # accessing pairs of candidate docs from LSH function output
            tup = ()
            for k,l in true_articles:                           # accessing pairs of doc ids in 10000 truth file
                if (id1 == k and id2 == l and id1 == i and id2 == j):
                    tup = tuple([id1]) + tuple([id2]) + tuple(['Yes']) + tuple(['Yes'])
                    true_positive = true_positive + 1
                    result.append(tup)
                elif (id1 != k and id2 != l and id1 == i and id2 == j):
                    tup = tuple([id1]) + tuple([id2]) + tuple(['No']) + tuple(['Yes'])
                    false_positive = false_positive + 1
                    result.append(tup)
                elif (id1 == k and id2 == l and id1 != i and id2 != j):
                    tup = tuple([id1]) + tuple([id2]) + tuple(['Yes']) + tuple(['No'])
                    false_negative = false_negative + 1
                    result.append(tup)
                elif (id1 != k and id2 != l and id1 != i and id2 != j):
                    tup = tuple([id1]) + tuple([id2]) + tuple(['No']) + tuple(['No'])
                    true_negative = true_negative + 1
                    result.append(tup)
    df = pd.DataFrame(result, columns=['ID1', 'ID2', 'True Label','LSH Label'])
    precision = true_positive/(true_positive + false_positive)
    recall = true_positive/(true_positive + false_negative)
    return precision, recall
    
                    
                    
    
    
    
    
    
                
                
            
                
        
            
            
        
            
    

ModuleNotFoundError: No module named 'plagiarism_lib'